In [ ]:
import altair as alt
import pandas as pd

# https://altair-viz.github.io/gallery/simple_bar_chart.html
# source = pd.DataFrame({
#     'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
#     'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
# })

# alt.Chart(source).mark_bar().encode(
#     x='a',
#     y='b'
# )

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)

In [ ]:
# inheriting usage notes from my other code for now (not self-plagiarization)
# https://github.com/cephcyn/cse517project/blob/master/embed_w2v.py
# for word2vec code; see also word2vec documentation
import gensim

# Load Google's pre-trained Word2Vec model.
# model source: https://code.google.com/archive/p/word2vec/
word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    'model/GoogleNews-vectors-negative300.bin', 
    binary=True)

In [ ]:
# Get the word2vec embedding of a phrase
def get_phrase_embed_word2vec(word2vec, phrase):
    try:
        phraseS = phrase.split()
    except:
        return pd.DataFrame()
    unknowns = []
    emb = []
    for w in phraseS:
        try:
            emb.append(word2vec[w])
        except:
            unknowns.append(w)
    if len(emb) == 0:
        return pd.DataFrame()
    emb_sum = pd.DataFrame(emb).sum()
    emb_sum['word'] = phrase
    return pd.DataFrame([emb_sum])

v = get_phrase_embed_word2vec(
    word2vec, 
    'test sentence')
sent_v = get_phrase_embed_word2vec(
    word2vec, 
    'This is a test sentence !')
v

In [ ]:
def embed_subtract(v, sent_v, dim):
    try:
        inverse_v = sent_v.iloc[:, 0:dim].subtract(v.iloc[:, 0:dim])
        inverse_v['word'] = v['word']
        inverse_v['sentence'] = sent_v['word']
        return inverse_v
    except:
        return pd.DataFrame()

embed_subtract(v, sent_v, 300)

In [ ]:
import spacy
from spacy.lang.en import English
from spacy import displacy

# OPTIONAL - to disable outputs from Tensorflow
import logging
logging.getLogger('tensorflow').disabled = True

# This line only needs to be a one-time download
# !python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_md')

import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import tensorflow_hub as hub

# Load ELMo model
url = "https://tfhub.dev/google/elmo/2"
elmo = hub.Module(url)

In [ ]:
# Get the ELMo embedding of a phrase (with given span limits)
def get_phrase_embed_elmo(elmo, sentence, span, phrase, pregenerated=None):
    # Do a NaN / None check
    # if phrase is not None, then span should not be empty...
    try:
        phraseS = phrase.split()
    except:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    # Do an empty string check
    if len(phraseS) == 0:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    # Generate or retrieve the ELMo embeddings for this sentence
    if pregenerated is None:
        # If we don't have a handy pre-generated dict of sentence:vector already...
        embeddings = elmo(
            [sentence], 
            signature='default', 
            as_dict=True)['elmo']
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            x = sess.run(embeddings)
            x = x[0]
    else:
        # otherwise just grab the vector
        x = pregenerated[sentence]
    # Get the ELMo embeddings for the phrase(s) / span(s) pieces that we asked for
    component_emb = None
    for i in range(len(span)):
        span0 = len(sentence[:int(span[i][0])].split(' ')) - 1
        span1 = len(sentence[:int(span[i][1])].split(' '))
        emb_sum = pd.DataFrame(x[span0:span1]).sum()
        emb_sum['word'] = phraseS[i]
        component_emb = component_emb.append(
            pd.DataFrame([emb_sum]), 
            ignore_index=True) if (component_emb is not None) else pd.DataFrame([emb_sum])
    # Get the sum ELMo embedding for entire phrase that was requested
    emb_sum = component_emb.sum()
    emb_sum['word'] = phrase
    # Get the ELMo embedding for the entire sentence (sum over the entire sentence)
    sentence_sum = pd.DataFrame(x).sum()
    sentence_sum['word'] = sentence
    return pd.DataFrame([emb_sum]), pd.DataFrame([sentence_sum]), component_emb

# v, sent_v, each_v = get_phrase_embed_elmo(
#     elmo, 
#     'This is a test sentence !',
#     [(10, 14), (15, 23)], 'test sentence')
# v

In [ ]:
# embed_subtract(v, sent_v, 1024)

In [ ]:
search_word = 'BERT'
anchor_type = 'coreference'

csv = pd.read_csv(
    f'outputs/{search_word}/{anchor_type}.csv', 
    index_col='Unnamed: 0')
csv.fillna('', inplace=True)
csv

In [ ]:
# output_w2v = csv.groupby(
#     csv.index, 
#     group_keys=True, 
#     as_index=False,
#     sort=True
# ).apply(
#     lambda group: get_phrase_embed_word2vec(
#         word2vec,
#         group.iloc[0]['c_verb_full']
#     )
# ).reset_index(level=1, drop=True)

output_w2v = csv.groupby(
    csv.index, 
    group_keys=True, 
    as_index=False,
    sort=True
).apply(
    lambda group: get_phrase_embed_word2vec(
        word2vec,
        group.iloc[0]['c_subj_full'] + (
            (' ' + group.iloc[0]['c_verb_full']) if group.iloc[0]['c_verb_full'] != '' else '')
    )
).reset_index(level=1, drop=True)

output_w2v

In [ ]:
# Centroid is the arithmetic mean position of all points in the figure
output_w2v_c = pd.DataFrame(
    [np.mean(output_w2v.iloc[:, 0:300])])
output_w2v_c['word'] = "[CENTROID]"
output_w2v_c

In [ ]:
output_w2v_inv = csv.groupby(
    csv.index, 
    group_keys=True, 
    as_index=False,
    sort=True
).apply(
    lambda group: embed_subtract(
        get_phrase_embed_word2vec(
            word2vec,
            group.iloc[0]['c_verb_full']
        ), 
        get_phrase_embed_word2vec(
            word2vec,
            ' '.join(group.iloc[0]['split_tokens'])
        ), 300)
).reset_index(level=1, drop=True)
output_w2v_inv

In [ ]:
import pickle
import ast

# save ELMo outputs so rerunning doesn't take forever...
# don't rerun this if we don't need to regenerate the ELMo outputs :|
all_sentences = [' '.join(ast.literal_eval(r)) for r in csv['split_tokens']]
embeddings = elmo(
    all_sentences, 
    signature='default', 
    as_dict=True)['elmo']
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    x = sess.run(embeddings)

x = dict(zip(all_sentences, x))
pickle.dump(x, open("temp/BERT_ELMo.pkl", "wb"))

In [ ]:
import ast
import pickle

x = pickle.load(open("temp/BERT_ELMo.pkl", "rb"))

# # Get ELMo embeddings for d_anchorverb
# output_elmo = csv.groupby(
#     csv.index, 
#     group_keys=True, 
#     as_index=False,
#     sort=True
# ).apply(
#     lambda group: get_phrase_embed_elmo(
#         elmo,
#         ' '.join(ast.literal_eval(group.iloc[0]['split_tokens'])),
#         [(group.iloc[0]['d_averb_span0'], group.iloc[0]['d_averb_span1'] - 1)],
#         group.iloc[0]['d_averb'], pregenerated=x
#     )[0]
# ).reset_index(level=1, drop=True)

# # Get ELMo embeddings for c_verb
# output_elmo = csv.groupby(
#     csv.index, 
#     group_keys=True, 
#     as_index=False,
#     sort=True
# ).apply(
#     lambda group: get_phrase_embed_elmo(
#         elmo,
#         ' '.join(ast.literal_eval(group.iloc[0]['split_tokens'])),
#         ast.literal_eval(group.iloc[0]['c_verb_allspans']),
#         group.iloc[0]['c_verb_full'], pregenerated=x
#     )[0]
# ).reset_index(level=1, drop=True)

# Get ELMo embeddings for a combination of c_subj and c_verb
output_elmo = csv.groupby(
    csv.index, 
    group_keys=True, 
    as_index=False,
    sort=True
).apply(
    lambda group: get_phrase_embed_elmo(
        elmo,
        ' '.join(ast.literal_eval(group.iloc[0]['split_tokens'])),
        ast.literal_eval(group.iloc[0]['c_subj_allspans']) + ast.literal_eval(group.iloc[0]['c_verb_allspans']),
        group.iloc[0]['c_subj_full'] + (
            (' ' + group.iloc[0]['c_verb_full']) if group.iloc[0]['c_verb_full'] != '' else ''), pregenerated=x
    )[0]
).reset_index(level=1, drop=True)

output_elmo

In [ ]:
# Centroid is the arithmetic mean position of all points in the figure
output_elmo_c = pd.DataFrame(
    [np.mean(output_elmo.iloc[:, 0:300])])
output_elmo_c['word'] = "[CENTROID]"
output_elmo_c

In [ ]:
import ast

def elmo_inv(row, pregenerated=None):
    v, sent_v, each_v = get_phrase_embed_elmo(
        elmo,
        ' '.join(ast.literal_eval(row['split_tokens'])),
        [(row['d_averb_span0'], row['d_averb_span1'] - 1)],
        row['d_averb'], pregenerated=pregenerated
    )
    sent_v_inv = embed_subtract(v, sent_v, 1024)
    return sent_v_inv

x = pickle.load(open("temp/BERT_ELMo.pkl", "rb"))
output_elmo_inv = csv.groupby(
    csv.index, 
    group_keys=True, 
    as_index=False,
    sort=True
).apply(
    lambda group: elmo_inv(group.iloc[0], pregenerated=x)
).reset_index(level=1, drop=True)

output_elmo_inv

In [ ]:
# Visualize the vectors
# loaned from :
# https://github.com/cephcyn/ChatlogGrapher/blob/master/data_processing.ipynb

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import altair as alt
alt.renderers.enable('default')

def visualize_embeds(data, reference, color=None, tooltip=['word']):
    x = data.iloc[:, 0:300]
    x = StandardScaler().fit_transform(x)

    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(
        data=principalComponents,
        columns=['pc1', 'pc2'])

    finalDf = principalDf
    finalDf = finalDf.set_index(data.index)
    finalDf['word'] = data['word']
    finalDf = finalDf.join(
        reference, 
        how='inner',
        lsuffix='_embed', 
        rsuffix='_ref'
    )

    chart = alt.Chart(finalDf).mark_circle(size=60)
    # should figure out a more pythonic way to do this :/
    if color is None:
        chart = chart.encode(
            x='pc1',
            y='pc2',
            tooltip=tooltip
        )
    else:
        chart = chart.encode(
            x='pc1',
            y='pc2',
            color=color,
            tooltip=tooltip
        )
    return chart.interactive()

# visualize_embeds(pd.concat([output_w2v, output_w2v_c]).reset_index())

In [ ]:
visualize_embeds(output_w2v, csv)

In [ ]:
visualize_embeds(output_w2v_inv, csv, tooltip=['word', 'Text'])

In [ ]:
visualize_embeds(output_elmo, csv)

In [ ]:
visualize_embeds(output_elmo_inv, csv, tooltip=['word', 'Text'])

In [ ]:
# this is the automatically-identified "c_verb_full"
output_manualgroups = pd.DataFrame(
    [['[introduce]', 'introduce'],
     ['[introduce]', 'present'],
     ['[introduce]', 'propose'],
     ['[success]', 'have advanced'],
     ['[success]', 'have achieved'],
     ['[success]', 'have pushed'],
     ['[success]', 'has improved'],
     ['[success]', 'has achieved'],
     ['[success]', 'has proven'],
     ['[success]', 'has shown'],
     ['[success]', 'has been applied'],
     ['[success]', 'has achieved'],
     ['[success]', 'was found'],
     ['[success]', 'is'],
     ['[success]', 'has yielded'],
     ['[successbut]', 'but'],
     ['[successbut]', 'remains'],
     ['[successbut]', 'have had'],
     ['[successbut]', 'have had'],
     ['[successbut]', 'have had'],
     ['[successbut]', 'explore'],
     ['[successbut]', 'but'],
     ['[successbut]', 'but'],
     ['[successbut]', 'but'],
     ['[stateproof]', 'show'],
     ['[introtask]', 'plays'],
     ['[introtask]', 'is'],
     ['[introtask]', 'are'],
     ['[introtask]', 'is'],
     ['[assessment]', 'assess'],
     ['[assessment]', 'studies'],
     ['[known]', 'allows'],
     ['[known]', 'are applied']])
output_manualgroups = output_manualgroups.rename(columns={0: "groupname", 1: "rootword"})

output_mg_v = output_manualgroups.groupby(
    output_manualgroups.index, 
    group_keys=True, 
    as_index=False,
    sort=True
).apply(
    lambda group: get_phrase_embed_word2vec(
        word2vec,
        group.iloc[0]['rootword']
    )
).reset_index(level=1, drop=True)

visualize_embeds(output_mg_v, output_manualgroups, color='groupname', tooltip=['word'])

In [ ]:
# Borrowed from
# https://gmarti.gitlab.io/ml/2017/09/07/how-to-sort-distance-matrix.html

from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline

N = len(output_w2v)
data = output_w2v.iloc[:, 0:300]
# data = output_elmo.iloc[:, 0:1024]
# Shuffle data for extra comparison
# data = data.sample(frac=1)
dist_mat = squareform(pdist(data))

plt.pcolormesh(dist_mat)
plt.colorbar()
plt.xlim([0,N])
plt.ylim([0,N])
plt.show()

In [ ]:
# Borrowed from
# https://gmarti.gitlab.io/ml/2017/09/07/how-to-sort-distance-matrix.html

from fastcluster import linkage

def seriation(Z,N,cur_index):
    '''
        input:
            - Z is a hierarchical tree (dendrogram)
            - N is the number of points given to the clustering process
            - cur_index is the position in the tree for the recursive traversal
        output:
            - order implied by the hierarchical tree Z
            
        seriation computes the order implied by a hierarchical tree (dendrogram)
    '''
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,method="ward"):
    '''
        input:
            - dist_mat is a distance matrix
            - method = ["ward","single","average","complete"]
        output:
            - seriated_dist is the input dist_mat,
              but with re-ordered rows and columns
              according to the seriation, i.e. the
              order implied by the hierarchical tree
            - res_order is the order implied by
              the hierarchical tree
            - res_linkage is the hierarchical tree (dendrogram)
        
        compute_serial_matrix transforms a distance matrix into 
        a sorted distance matrix according to the order implied 
        by the hierarchical tree (dendrogram)
    '''
    N = len(dist_mat)
    flat_dist_mat = squareform(dist_mat)
    res_linkage = linkage(flat_dist_mat, method=method,preserve_input=True)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage

In [ ]:
# Notes on the different clustering linkage methods: 
# https://en.wikipedia.org/wiki/Complete-linkage_clustering
# https://en.wikipedia.org/wiki/Hierarchical_clustering
# (The main difference is in how distances between clusters are calculated)

# Use Ward because it is generally best for "topic" clusters
# https://stats.stackexchange.com/questions/195446/choosing-the-right-linkage-method-for-hierarchical-clustering
methods = ["ward"]#,"single","average","complete"]
for method in methods:
    print("Method:\t",method)
    
    ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(dist_mat,method)
    
    plt.pcolormesh(ordered_dist_mat)
    plt.xlim([0,N])
    plt.ylim([0,N])
    plt.show()

In [ ]:
data = output_w2v.join(
    csv, 
    how='inner'
)
data = data.loc[csv['c_subj_split'] != ''].loc[csv['c_verb_split'] != '']
# Shuffle data?
# data = data.sample(frac=1)
dist_mat = squareform(pdist(data.iloc[:, 0:300]))
# dist_mat = squareform(pdist(data.iloc[:, 0:1024]))

ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(
    dist_mat,
    'ward')
reordered_data = data
reordered_data['temp_index'] = data.index
reordered_data = reordered_data.reset_index(drop=True)
reordered_data = reordered_data.iloc[res_order].reset_index(drop=True)
reordered_data['order_cluster'] = reordered_data.index
reordered_data = reordered_data.set_index('temp_index')

reordered_data = csv.join(
    reordered_data['order_cluster'], 
    how='outer'
)
reordered_data.to_csv(f'temp/BERT_csubjverb_w2v_ordercluster.csv')
reordered_data

In [ ]:
# https://github.com/cephcyn/ChatlogGrapher/blob/master/data_processing.ipynb
# for cosine similarity; see also sklearn documentation
from sklearn.metrics.pairwise import cosine_similarity

# Compute a distance metric column from some origin phrase and given some column name to calculate diff of
def distance_word2vec(row, origin, colname):
    try:
        sim = cosine_similarity(get_phrase_vector(origin), get_phrase_vector(row[colname]))[0][0]
    except:
        sim = -1
    return dict(zip(['distance'], [sim]))

base_averb = 'encodes'
output = csv.apply(
    lambda row: distance_word2vec(
        row, 
        base_averb, 
        'averb'
    ), 
    axis=1, result_type='expand')

output = csv.join(output).sort_values(by=['distance'], ascending=False)
output.to_csv(f'temp/BERT_similarity_encodes.csv')
output

In [ ]:
# LDA clustering for the sentence structure lists
# We want to take into account both the elements of the structure lists, and ALSO the ordering

# TODO
# reference https://github.com/cephcyn/cse517project
# reference https://radimrehurek.com/gensim/models/ldamodel.html

# from sklearn.feature_extraction.text import CountVectorizer
# import lda

# l = [' '.join(i) for i in [A,B,C,D,E,F]]
# vec = CountVectorizer(analyzer='word', ngram_range=(1,1))

# X = vec.fit_transform(l)

# model = lda.LDA(n_topics=2, random_state=1)
# model.fit(X)

# doc_topic = model.doc_topic_

# for i in range(len(l)):
#     print(f'Cluster {i}: Topic ', doc_topic[i].argmax())